In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import multiprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

from scipy.misc import imread, imresize
import imageio
import skimage

/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
!pwd

/Users/Mao/Documents/fall_mod2_18/dl/project


In [6]:
! ls /Users/Mao/Documents/fall_mod2_18/dl/project/data/imgs

test  train


In [7]:
labels = 'c0,c1,c2,c3,c4,c5,c6,c7,c8,c9'.split(',')
def get_train():
    "Get the training data into a DataFrame"
    root = os.path.join(os.getcwd(), 'data', 'imgs')
    labels = [i for i in os.listdir(os.path.join(root,'train')) if not i.startswith(".")]
    labels.sort()
    print('Found labels: ', *labels)
    data = []
    for lab in labels:
        paths = os.listdir(os.path.join(root,'train', lab))
        X = [(os.path.join(root, 'train', lab, i), lab) for i in paths]
        data.extend(X)
    import random
    random.shuffle(data) # since labels were sorted
    df = pd.DataFrame({'paths': [i[0] for i in data],
                       'target': [i[1] for i in data]})

    for cl in labels:
        df[cl] = df.target == cl
    df.drop('target', 1, inplace=True)
    return df

In [8]:
train = get_train().sample(2000)

Found labels:  c0 c1 c2 c3 c4 c5 c6 c7 c8 c9


In [7]:
train.head

<bound method NDFrame.head of                                                    paths     c0     c1     c2  \
6663   /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False   
9679   /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False   
19866  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False   
15573  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...   True  False  False   
19123  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False   
11111  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False   
12729  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False   True  False   
18821  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False   True  False   
12066  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False   
10865  /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False   True   
2049   /Users/yu_yun/Desktop/MSAN/kaggle-distracted-d...  False  False  False  

In [9]:
def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = multiprocessing.cpu_count()
    print("Using", workers, "workers to apply function")
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def getimage(x):
    return imresize(imread(x, 'L'), (100, 100)).flatten()

In [10]:
train['images'] = apply_by_multiprocessing(train.paths, getimage) 
X = np.array([i for i in train.images])
print('Loaded training data')

Using 4 workers to apply function


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`

Loaded training data


### Train Classifier

In [11]:
classifiers = [RandomForestClassifier(n_jobs=-1,
                                      n_estimators=100
                                      ) for i in labels]
targets = [train[i] for i in labels]


In [12]:
for i, clf, Y in zip(labels, classifiers, targets):
    #print(cross_val_score(clf, X, Y, scoring='log_loss'))
    clf.fit(X, Y)
    print('Trained', i)
print('Done training')

Trained c0
Trained c1
Trained c2
Trained c3
Trained c4
Trained c5
Trained c6
Trained c7
Trained c8
Trained c9
Done training


In [16]:
def get_test():
    "Get test data into a DataFrame"
    
    root = os.path.join(os.getcwd(), 'data', 'imgs')
    paths = os.listdir(os.path.join(root, 'test'))
    x = [os.path.join(root, 'test', i) for i in paths]
    x.sort()
    df = pd.DataFrame({'paths': x,
                       })
    return df

In [17]:
print('loading images')
test = get_test()

loading images


#### Prediciton

In [19]:
test = get_test()
test['images'] = apply_by_multiprocessing(test.paths, getimage)
X = np.array([i for i in test.images])
print('Done loading test data.')
print("Running predictions")
results = []
for index, clf in enumerate(classifiers):
    results.append(clf.predict_proba(X)[:,1])
    print('c'+str(index), 'Done with prediction')

print('Creating submission')
c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 = results
sub = pd.DataFrame({'img': test.paths.apply(lambda x:x.split('/')[-1]),
                    'c0': c0,'c1': c1,'c2': c2,
                    'c3': c3,'c4': c4,'c5': c5,
                    'c6': c6,'c7': c7,'c8': c8,
                    'c9': c9,})
sub[['img', 'c0', 'c1', 'c2', 'c3',
     'c4', 'c5', 'c6', 'c7', 'c8', 'c9']].to_csv('submission.csv', index=False)
print('Done!')


Using 4 workers to apply function


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`

Done loading test data.
Running predictions
c0 Done with prediction
c1 Done with prediction
c2 Done with prediction
c3 Done with prediction
c4 Done with prediction
c5 Done with prediction
c6 Done with prediction
c7 Done with prediction
c8 Done with prediction
c9 Done with prediction
Creating submission
Done!
